Bayes Classifier- spam & not spam

In [39]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [9]:
#importing data
spamtr=[open('spam-train/spamtrain{}.txt'.format(i)).read() for i in range(349)] #350 file
nospamtr=[open('nonspam-train/nonspamtrain{}.txt'.format(i)).read() for i in range(349)]
spamtest=[open('spam-test/spam{}.txt'.format(i)).read() for i in range(129)]
nospamtest=[open('nonspam-test/nonspam{}.txt'.format(i)).read() for i in range(129)]

In [15]:
#The data is now stored in 4 lists. See this for example
spamtr[0]
#nospamtr[1:3]

'great parttime summer job display box credit application need place small owneroperate store area here introduce yourself store owner manager our effective script tell little display box save customer hundred dollar draw card business every app send spot counter place box nothing need need name address company send commission check compensaation every box place become representative earn commission each application store course much profitable plan pay month small effort call code hours receive detail removed our mailing list type b hotmail com area remove subject area e mail send '

Next, we should tokenize these into a meaningful unit, such as words. There are several method for tokenizing words. For example, we can use word_tokenize from nltk. Or we can use find() function. Note that find() is better here becasue it handles punctuation as well. see appendix (end of the notebook) for examples.

We want to know in how many documents each word has occured (not how many times total or how many times per document). So we iterate over documents and we use "set" to count multiple occurances in each document only once. (see appendix for details)

We will define two counters. One for words in spam documents and one for words in non spam documents. We also use np.sum instead of regular sum becasue regular sum doesn't support sum for counter objects. 


In [45]:
import numpy as np

In [71]:
n_sp = np.sum([Counter(set(re.findall("[a-z0-9']+",txt.lower()))) for txt in spamtr])
n_nonsp = np.sum([Counter(set(re.findall("[a-z0-9']+",txt.lower()))) for txt in nospamtr])

In [72]:
#see what n_sp is (a dictionary with counts)
#n_sp 
#n_sp["month"] # we can get the count this way

we can see most common words using most_common() function from counter. In this data set seems like words such as "a, the, an" have already been removed. If it wasn't the case, we'd need to ignore top 10-20 words or all those words that occur in all documents becasue they don't provide any useful information for classification. 

In [73]:
#n_sp.most_common()[:10] #top 10 most common words in all spam emails

Let's look at Bayes formula:

$$
p({\rm spam}\ |\ words) = \frac{p(words\ |\ {\rm spam})\,p({\rm spam})}
{p(words\ |\ {\rm spam})\,p({\rm spam})+p(words\ |\ {\rm nonSpam})\,p({\rm nonSpam})}
\approx\frac{\prod_i p(w_i\ |\ {\rm spam})}{\prod_i p(w_i\ |\ {\rm spam})+\prod_i p(w_i\ |\ {\rm nonSpam})}
$$

Note: For simplicity, we assume p(spam)=P(not spam)= 0.5 here.
(The second line says we multiply this for each word)

What does the formula say? 
It says that for each word in the "unknown"/test document, first we take a look at the probability of that word in both train sets (spam train set and non spam train set). For example, if the word "award" occured in 200 documents out of 350 spam trainset then P("award"|spam)=200/350. Similarly we get a P("award"|nonSpam)).
The unknown document has several words and we want to look at all the common words in it. It's a choice on how many common words we keep but adding too many could add noise. 
For each word we calculate the P(word|spam) and P(word|nonspam) and calculate this which is a multiplication:

$$
\approx\frac{\prod_i p(w_i\ |\ {\rm spam})}{\prod_i p(w_i\ |\ {\rm spam})+\prod_i p(w_i\ |\ {\rm nonSpam})}
$$





In [116]:
def Bayes(txt):
    
    #finds the word count in unknown doc txt
    wc=Counter(re.findall("[a-z0-9']+", txt.lower()))
    # keeping only 20 common words
  
    kw=[w for w,c in wc.most_common(15)]
    
    
    t=1
    k=1
    for w in kw:
        if w not in n_sp: n_sp[w]=0.1 # smoothing see appendix
        if w not in n_nonsp: n_nonsp[w]=0.1
        
        t *= n_sp[w]/350
        k *=n_nonsp[w]/350
             

    spamProbability= t/(t+k)
    
    if(spamProbability>=0.5):
        print("it's probably spam", spamProbability)
    else:
        print("it's probably not spam", spamProbability)
        
        

See the example below, correctly identifies all the emails that were "not spam"

In [1]:
for i in range(100): # first 10 docs in the test set
    Bayes(nospamtest[i])

Correctly detects 9 out of 10 spam emails.

In [2]:
for i in range(10):
    Bayes(spamtest[i])

We can tell that this very simple classifier works relatively well and there are ways for improving this which is a topic for later :)

Appendix

See the example below (here's vs here & 's) for comparison of word tokenizer from nltk and find() function

In [36]:
from nltk.tokenize import sent_tokenize, word_tokenize
word_tokenize("Here's a test of word_tokenizer and find function!")

['Here',
 "'s",
 'a',
 'test',
 'of',
 'word_tokenizer',
 'and',
 'find',
 'function',
 '!']

In [37]:
re.findall("[a-z0-9'_]+","Here's a test of word_tokenizer and find function!".lower())
#return all the words as a list

["here's", 'a', 'test', 'of', 'word_tokenizer', 'and', 'find', 'function']

Counter is very useful as it keeps the counts in a dictionary for us. By adding the "set" we count all the occurances of a word in a document only once becasue we are itnerested in number of documents that has a word not the frequency of words per document.

In [41]:
Counter(set(re.findall("[a-z0-9'_]+",spamtest[0].lower()))) # with set

In [43]:
Counter((re.findall("[a-z0-9'_]+",spamtest[0].lower()))) # with no set

Smoothing

For words that are in the test document but haven't been in train set we need to do an adjustment. We set the number to 0.1 (as if the word occured in 0.1 of documents) to avoid multiplying by zero and losing data. This is callled "smoothing".